## Группа DS03-onl

Студент Парфимович Алексей

## Домашнее задание №9

### Работа с изображениями с целью аугментации.
    
1. Скачать изображения, на которых автомобили есть или нет: 100 изображений целевого класса и 100 нецелевого.
2. Выполнить аугментацию (вращение, растяжение/сжатие, увеличение/уменьшение) полученных изображений - количество изображений должно увеличиться минимум в 3 раза (600 шт), размер целевых изображений 256x256px.
3. Использовать библиотеки: OpenCV, pillow, scikit-image, tf.image. 

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import random_noise

SRC_DIR = 'src_img'
DST_DIR = 'dest_img'
DEST_IMG_SIZE = 256

1. Разрезать изображение на части требуемого размера, если это позволяют его размеры
Иначе, можно обрезать изображение до требуемого размера по случайным координатам

In [2]:
def cut_img(img_name, h_size, w_size):
    
    '''Cuts the source image into parts h_size*w_size px. If not possible - crops the image randomly according to the specified size '''
    
    img_path = f'{SRC_DIR}\{img_name}'
    img = cv2.imread(img_path)

    # Оперделить возможное количество частей изображения
    (h,w) = img.shape[0:2]
    h_num = h//h_size
    w_num = w//w_size

    img_num = 1
    img_name = img_name[:-4]
    for y in range(h_num):
        if h_num > 1: y = y * h_size
        # Если изображение не достаточно большое по высоте - выбрать случайную координату для обрезки
        else: y = np.random.randint(0, h - h_size)

        for x in range(w_num):
            if w_num > 1: x = x * w_size
            # Если изображение не достаточно большое по ширине - выбрать случайную координату для обрезки
            else: x = np.random.randint(0, w - w_size)

            # Вырезать часть изображения
            cropped = img[y:y+h_size, x:x+w_size]
            # Сохранить фрагмент в файл
            cv2.imwrite(f'{DST_DIR}\{img_name}_cut{img_num}.jpg', cropped)
            img_num += 1

2. Выполнить поворот изображения на заданный угол

In [3]:
def rotate_img(image, angle=90, scale=1.0):
        '''
        Rotate the image
        :param image: Image to be processed
        :param angle: Rotation angle in degrees.
        :param scale: Scale factor.
        '''
        (h,w) = image.shape[0:2]

        # Сформировать матрицу вращения
        M = cv2.getRotationMatrix2D((w//2,h//2), angle, scale)
        # ВЫполнить поворот изображения
        image = cv2.warpAffine(image,M,(w,h))
        
        return image

3. Развернуть изображение зеркально по горизонтали и/или вертикали

In [4]:
def flip_img(image, vflip=False, hflip=False):
        '''
        Flip the image
        :param image: Image to be processed
        :param vflip: Flip the image vertically
        :param hflip: Flip the image horizontally
        '''
        if hflip or vflip:
            if hflip and vflip:
                c = -1
            else:
                c = 0 if vflip else 1
            image = cv2.flip(image, flipCode=c)
        return image 

4. Обрезать изображение по заданному размеру

In [5]:
def crop_img(image, h_size, w_size ):
    '''
    Crop the image
    :param image: Image to be processed
    :param h_size: Target height 
    :param w_size: Target width
    '''
    (h,w) = image.shape[0:2]

    # Определение координат
    y = h//2 - h_size//2
    x = w//2 - w_size//2
    # Обрезка изображения
    return image[y:y+h_size, x:x+w_size]


5. Сжатие изображения до заданных размеров

In [6]:
def resize_img(image, h_size, w_size ):
    '''
    Resize the image
    :param image: Image to be processed
    :param h_size: Target height 
    :param w_size: Target width
    '''
    (h,w) = image.shape[0:2]

    ratio = h/w

    if ratio < 1:
        dimension = (w_size, int(h_size*ratio))
    else:
        dimension = (int(w_size*ratio), h_size)

    return cv2.resize(image, dimension)

6. Добавление шума на изображение

In [7]:
def noise_img(image, mode='s&p', amount=0.011):
    
    noise = random_noise(image, mode=mode, amount=amount)

    # Приведенная выше функция возвращает значения с плавающей запятой в диапазоне [0, 1]
    # требуется изменить его на «uint8» в диапазоне [0,255]
    return np.array(255 * noise, dtype=np.uint8)

In [8]:
'''
src_img_list = os.listdir(SRC_DIR)

img_path = f'{SRC_DIR}\{src_img_list[0]}'
img = cv2.imread(img_path)

img2 = resize_img(img, DEST_IMG_SIZE, DEST_IMG_SIZE)
plt.imshow(img2)
'''

"\nsrc_img_list = os.listdir(SRC_DIR)\n\nimg_path = f'{SRC_DIR}\\{src_img_list[0]}'\nimg = cv2.imread(img_path)\n\nimg2 = resize_img(img, DEST_IMG_SIZE, DEST_IMG_SIZE)\nplt.imshow(img2)\n"

### Выполним аугментацию исходных изображений

In [9]:
src_img_list = os.listdir(SRC_DIR)

for img_name in src_img_list:

    # Разрежем сообщения на части, если это возможно, и сохраним части в отдельные файлы
    cut_img(img_name, DEST_IMG_SIZE, DEST_IMG_SIZE)

    img_path = f'{SRC_DIR}\{img_name}'
    img = cv2.imread(img_path)

    # Уменьшим исходное изображение
    resized = resize_img(img, DEST_IMG_SIZE, DEST_IMG_SIZE)
    cv2.imwrite(f'{DST_DIR}\{img_name}_resised.jpg', resized)

    # Зеркально развернем изображение по горизонтали
    flipped = flip_img(resized, hflip=True)
    cv2.imwrite(f'{DST_DIR}\{img_name}_hflipped.jpg', flipped)

    # Добавим на исходное изображение шум
    noized = noise_img(img, amount=0.1)
    resized = resize_img(noized, DEST_IMG_SIZE, DEST_IMG_SIZE)
    cv2.imwrite(f'{DST_DIR}\{img_name}_noized.jpg', resized)

    # Повернем исходное изображение на 90 градусов и уменьшим его
    rotated = rotate_img(img)
    resized = resize_img(rotated, DEST_IMG_SIZE, DEST_IMG_SIZE)
    cv2.imwrite(f'{DST_DIR}\{img_name}_rotated.jpg', resized)